<a href="https://colab.research.google.com/github/Saldana20021917/Map.github.io/blob/main/Banco_Mundial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

# Indicadores
indicadores = {
    "NY.GDP.MKTP.CD": "PIB (USD a precios actuales)",
    "NY.GDP.PCAP.CD": "PIB per cápita (USD actuales)",
    "NY.GDP.MKTP.KD.ZG": "Crecimiento del PIB (% anual)",
    "NE.GDI.FTOT.ZS": "Formación bruta de capital fijo (% PIB)",
    "FP.CPI.TOTL.ZG": "Inflación (IPC, % anual)",
    "NE.EXP.GNFS.ZS": "Exportaciones (% del PIB)",
    "NE.IMP.GNFS.ZS": "Importaciones (% del PIB)",
    "BN.GSR.GNFS.CD": "Balanza comercial (USD actuales)",
    "TM.TAX.MANF.WM.AR.ZS": "Tasa arancelaria promedio ponderada (%)",
    "SL.EMP.TOTL.SP.ZS": "Tasa de empleo (% población)",
    "SL.UEM.TOTL.ZS": "Tasa de desempleo (% fuerza laboral)",
    "SI.POV.NAHC": "Pobreza (% bajo la línea nacional)",
    "NY.GNP.PCAP.PP.CD": "Ingreso per cápita (PPP, USD internacionales)"
}

# Años
años = "2000:2024"

# Países de América del Sur (ISO-3)
paises_suramerica = ["ARG", "BOL", "BRA", "CHL", "COL", "ECU", "GUY", "PRY", "PER", "SUR", "URY", "VEN"]

# Lista para almacenar resultados
data_list = []

# Iterar sobre cada indicador y país
for codigo, nombre in indicadores.items():
    url = f"https://api.worldbank.org/v2/country/{';'.join(paises_suramerica)}/indicator/{codigo}?date={años}&format=json&per_page=20000&language=es"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if len(data) > 1 and isinstance(data[1], list):
            df = pd.DataFrame(data[1])
            df = df[['country', 'date', 'value']]
            df['country'] = df['country'].apply(lambda x: x['value'])
            df = df.rename(columns={'value': nombre})
            data_list.append(df)
        else:
            print(f"No se encontraron datos para {nombre} ({codigo})")
    else:
        print(f"Error al obtener {nombre} ({codigo}): {response.status_code}")

# Unir todos los DataFrames por país y año
if data_list:
    df_final = data_list[0]
    for df in data_list[1:]:
        df_final = pd.merge(df_final, df, on=['country', 'date'], how='outer')

    # Exportar a Excel
    df_final.to_excel("datos_america_del_sur.xlsx", index=False)
    print("Archivo guardado como 'datos_america_del_sur.xlsx'")
else:
    print("No se obtuvieron datos para ningún indicador.")


# Consumir API del Banco Mundial







**1. Importación de Librerías**

In [1]:
import requests
import pandas as pd


**Se importan las librerías necesarias:**

**requests:** Permite realizar solicitudes HTTP para obtener datos desde la API del Banco Mundial.

**pandas:** Librería para manipulación y análisis de datos en formato tabular, ideal para organizar la información recibida.

**2. Crear diccionario de indicadores necesarios**

In [2]:
indicadores = {
    "NY.GDP.MKTP.CD": "PIB (USD a precios actuales)",
    "NY.GDP.PCAP.CD": "PIB per cápita (USD actuales)",
    "NY.GDP.MKTP.KD.ZG": "Crecimiento del PIB (% anual)",
    "NE.GDI.FTOT.ZS": "Formación bruta de capital fijo (% PIB)",
    "FP.CPI.TOTL.ZG": "Inflación (IPC, % anual)",
    "NE.EXP.GNFS.ZS": "Exportaciones (% del PIB)",
    "NE.IMP.GNFS.ZS": "Importaciones (% del PIB)",
    "BN.GSR.GNFS.CD": "Balanza comercial (USD actuales)",
    "TM.TAX.MANF.WM.AR.ZS": "Tasa arancelaria promedio ponderada (%)",
    "SL.EMP.TOTL.SP.ZS": "Tasa de empleo (% población)",
    "SL.UEM.TOTL.ZS": "Tasa de desempleo (% fuerza laboral)",
    "SI.POV.NAHC": "Pobreza (% bajo la línea nacional)",
    "NY.GNP.PCAP.PP.CD": "Ingreso per cápita (PPP, USD internacionales)"
}

El diccionario almacena los códigos de los indicadores como claves y sus descripciones como valores, facilitando la identificación posterior de los datos extraídos.

**3. Definición del Rango de Años**

In [3]:
años = "2000:2023"

Se define el rango de años de interés para consultar los datos, en este caso desde 2000 hasta 2023.


**4. Definimos los paises a utilizar**

Aqui debemos poner el ISO-3 de los paises a utilizar

In [4]:
paises_suramerica = ["ARG", "BOL", "BRA", "CHL", "COL", "ECU", "GUY", "PRY", "PER", "SUR", "URY", "VEN"]

**5. Lista para Almacenar Datos**

In [5]:
data_list = []

Se crea una lista vacía que almacenará los DataFrames de cada indicador para facilitar la combinación posterior.

**6. Iteración sobre los Indicadores**

In [16]:
for codigo, nombre in indicadores.items():
    url = f"https://api.worldbank.org/v2/country/{';'.join(paises_suramerica)}/indicator/{codigo}?date={años}&format=json&per_page=20000&language=es"
    response = requests.get(url)

Aqui recorremos el diccionario "indicadores" usando el método items(), que devuelve pares clave-valor. La variable codigo contiene el código del indicador del Banco Mundial (clave) y nombre su descripción (valor). Para cada par, se construye una URL que consulta los datos del indicador en todos los países de América del Sur definidos en la lista paises_suramerica.

La URL se forma utilizando una cadena formateada (f-string) con los siguientes elementos:

**country={';'.join(paises_suramerica)}:** Se incluyen los países a utilizar como una única cadena, separados por punto y coma, tal como lo exige la API para realizar solicitudes múltiples de países en una sola consulta.

**indicator={codigo}:** Se inserta el código del indicador correspondiente en cada iteración. Tambien se puede colocar "all" y se obtienen todos los paises disponibles para el bancon mundial.

**?date={años}:** Rango de años definido.

**&format**=json: Indica que la respuesta debe estar en formato JSON.

**&per_page=20000:** Establece el número máximo de registros por página.

**&language=es:** Solicita que los datos vengan en idioma español.



**7. Validación de Respuesta**

In [7]:
if response.status_code == 200:
    data = response.json()

La API devuelve códigos HTTP, donde 200 indica éxito. Si se recibe este código, se convierte la respuesta JSON a un objeto Python con json(), que facilita la manipulación de los datos.

**8. Verificación de Datos**

In [8]:
if len(data) > 1 and isinstance(data[1], list):
    df = pd.DataFrame(data[1])



La API devuelve los datos en una lista.

**len(data) > 1:** Asegura que haya más de un elemento.

**isinstance(data[1], list):** Comprueba que el segundo elemento sea una lista antes de convertirla en DataFrame con pd.DataFrame().

**9. Selección de Columnas Relevantes**

In [ ]:
df = df[['country', 'date', 'value']]
df['country'] = df['country'].apply(lambda x: x['value'])

Se seleccionan solo las columnas necesarias:

**country:** Nombre del país.

**date:** Año.

**value:** Valor del indicador.

La columna country contiene diccionarios, por lo que se aplica una función lambda con apply() para extraer el valor con la clave 'value'.

**10. Renombrar la Columna**

In [10]:
df = df.rename(columns={'value': nombre})



El método rename() cambia el nombre de la columna value por el nombre descriptivo del indicador (lo agregamos inicialmente en el diccionario), lo que facilita la interpretación.

**11. Almacenar el DataFrame**

In [11]:
data_list.append(df)

El DataFrame procesado se agrega a la lista para su posterior combinación.

**12. Mensajes de Error**

In [13]:
else:
  print(f" No se encontraron datos para {nombre} ({codigo})")

SyntaxError: invalid syntax (<ipython-input-13-3df1cc688c3b>, line 1)

Si no hay datos disponibles para el indicador, se genera un mensaje de advertencia para informar al usuario.

In [ ]:
else:
 print(f"Error al obtener {nombre} ({codigo}): {response.status_code}")

Este bloque se ejecuta cuando la respuesta de la API no fue exitosa.

**13. Fusión de DataFrames**

In [ ]:
if data_list:
    df_final = data_list[0]
    for df in data_list[1:]:
        df_final = pd.merge(df_final, df, on=['country', 'date'], how='outer')

Teniendo en cuenta que no nos salio ningun error. Realizamos un mergue osea un ( pegado) por lo cual se fusionan todos los DataFrames de la lista utilizando una unión externa (outer). Esto asegura que no se pierdan datos incluso si algunos indicadores no están disponibles para ciertos países o años.

**14. Exportación a Excel**

In [ ]:
  df_final.to_excel("datos_america_del_sur.xlsx", index=False)
  print("Archivo guardado como 'datos_america_del_sur.xlsx'")
else:
  print("No se obtuvieron datos para ningún indicador.")

Por ultimo se hace una exportacion, en este caso a excel, pero tambien se puede exportar en multiples formatos, y esta las alerta por si no se ejecuta el consumo de la API

# Conclusión

Este código permite automatizar la extracción de datos económicos y sociales desde la API del Banco Mundial, garantizando la obtención de información actualizada para múltiples países y años. Facilita la integración de diferentes indicadores en un solo archivo para su posterior análisis.



In [15]:
df_final

,country,date,PIB (USD a precios actuales),PIB per cápita (USD actuales),Crecimiento del PIB (% anual),Formación bruta de capital fijo (% PIB),"Inflación (IPC, % anual)",Exportaciones (% del PIB),Importaciones (% del PIB),Balanza comercial (USD actuales),Tasa arancelaria promedio ponderada (%),Tasa de empleo (% población),Tasa de desempleo (% fuerza laboral),Pobreza (% bajo la línea nacional),"Ingreso per cápita (PPP, USD internacionales)"
0,Argentina,2000,2.842038e+11,7637.014892,-0.788999,16.192649,NaN,10.986375,11.636070,-1.831883e+09,11.33,52.975,15.000,NaN,11190.0
1,Argentina,2001,2.686968e+11,7141.475077,-4.408840,14.179118,NaN,11.579008,10.273247,3.521811e+09,10.01,50.969,17.320,NaN,10800.0
2,Argentina,2002,9.772400e+10,2569.699635,-10.894485,11.960648,NaN,28.382597,13.370127,1.571728e+10,13.12,48.336,19.590,NaN,9450.0
3,Argentina,2003,1.275870e+11,3320.477751,8.837041,15.137508,NaN,25.930943,14.713805,1.561150e+10,14.05,51.646,15.360,NaN,10490.0
4,Argentina,2004,1.646579e+11,4242.020991,9.029573,15.893787,NaN,23.847619,16.845027,1.193361e+10,6.50,53.567,13.522,NaN,10770.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"Venezuela, RB",2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.91,48.038,7.530,NaN,NaN
296,"Venezuela, RB",2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.37,48.450,7.037,NaN,NaN
297,"Venezuela, RB",2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.81,49.747,5.707,NaN,NaN
298,"Venezuela, RB",2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.054,5.485,NaN,NaN
